In [1]:
# Importing Libraries
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError, KafkaTimeoutError

In [2]:
# Create spark session
spark = SparkSession.builder \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2') \
    .appName("Crypto Prediction") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")


In [3]:
# Create dataframe and read data from kafka topic
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe","CryptoPrediction2") \
    .option("startingOffsets","earliest") \
    .load()

In [4]:
# Transfrom dataframe by filtering the selected columns and dropping the remaining columns
df_upd = df.withColumn('value_str', df['value'].cast('string').alias('key_str')).drop('value') \
        .drop('key','topic','partition','offset','timestamp','timestampType')

In [6]:
# writing the datatframe to Hadoop directory and keep it running until terminated 
df_upd.writeStream \
  .outputMode("append") \
  .format("json") \
  .option("truncate","false") \
  .option("path", "C:\\Users\\Futurense\\OneDrive\\Desktop\\CryptoPrediction\\Realout") \
  .option("checkpointLocation","C:\\Users\\Futurense\\OneDrive\\Desktop\\CryptoPrediction\\Realout") \
  .start()